<a href="https://colab.research.google.com/github/Ron-Wu/Machine-learning-0602/blob/main/titanic1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from urllib.request import urlretrieve
url = "https://github.com/Ron-Wu/Machine-learning-0602/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Ron-Wu/Machine-learning-0602/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7fe479105550>)

In [16]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding = "utf-8")
test_df = pd.read_csv("test.csv", encoding = "utf-8")

In [17]:
# train_df  # 測試看是否有成功列出訓練資料
# test_df  # 測試看是否有成功列出測試資料

In [18]:
data = pd.concat([train_df, test_df], ignore_index = True)
data = data.drop(["PassengerId", "Survived"], axis = 1)

# data = pd.concat([train_df, test_df], ignore_index=True)
# data = data.drop(["PassengerId", "Survived"], axis=1)

In [19]:
na = data.isna().sum()
# series[帶入跟你的資料筆數一樣多True/False list]
na[na > 0].sort_values()
# na[na > 0].sort_values(ascending=False)  ## ascending=False 讓資料倒序

Fare           1
Embarked       2
Age          263
Cabin       1014
dtype: int64

In [21]:
def cabin_head(s):
  if not pd.isna(s):
    return s[0]
data["Cabin"] = data["Cabin"].apply(cabin_head)

0       None
1          C
2       None
3          C
4       None
        ... 
1304    None
1305       C
1306    None
1307    None
1308    None
Name: Cabin, Length: 1309, dtype: object

In [ ]:
dic = data["Ticket"].value_counts()
data["Ticket"] = data["Ticket"].apply(lambda t:dic[t])

In [27]:
# 補缺失值(類別：最常出現) [!如果一堆測試資料，不要重算，直接補Ｓ]
most = data["Embarked"].value_counts().idxmax() ## .idxmax()：取得最大值的索引，.idxmin()：取得最小值的索引
data["Embarked"] = data["Embarked"].fillna(most)
na = data.isna().sum()
na[na > 0].sort_values()

Fare        1
Age       263
Cabin    1014
dtype: int64

In [29]:
# 補缺失值，數值：補中位數
med = data.median().drop(["Pclass"])  # 因為Pclass非數字型態，所以要在中位數字中將其先去除
data = data.fillna(med)
na = data.isna().sum()
na[na > 0].sort_values()

Cabin    1014
dtype: int64